In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import seaborn as sns
from wordcloud import WordCloud

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
fake = pd.read_csv('.\\dataset\\fake.csv')
fake.head()

In [ ]:
fake['subject'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='subject',data=fake)

In [ ]:
## wordcloud -----------> for wordcloud we have to make signle text data. so need to join the list data.
text = ' '.join(fake['text'].tolist())

In [ ]:
wordcloud = WordCloud(width=1920, height=1080).generate(text)
fig  = plt.figure(figsize=(10, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
real = pd.read_csv(".\\dataset\\True.csv")
real.head()

In [ ]:
text = ' '.join(real['text'].tolist())
wordcloud = WordCloud(width=1920, height=1080).generate(text)
fig  = plt.figure(figsize=(10, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
unknown_publisher = []
for index, row in enumerate(real.text.values):
    try:
        record = row.split(' - ', maxsplit=1)
        assert(len(record[0]) < 260)   ### check the length of the reuters less then tweet data. tweet data length is 260. so if data is > 260 then consider as unknown publisher.
        assert(len(record) == 2)  ### this check is used for empty text. empty text is consider as unknown publisher
    except:
        unknown_publisher.append(index)

In [ ]:
len(unknown_publisher)

In [ ]:
temp_text = []
publisher = []

for index, row in enumerate(real.text.values):
    if index in unknown_publisher:
        publisher.append('Unknown')
        temp_text.append(row)
    else:
        record = row.split(' - ', maxsplit=1)
        publisher.append(record[0])
        temp_text.append(record[1])


In [ ]:
real['publisher'] = publisher
real['text'] = temp_text

In [ ]:
real.head()

Fake news analysis

In [ ]:
fake_empty_index = [index for index, text in enumerate(fake.text.tolist()) if str(text).strip() == ""]
len(fake_empty_index)

In [ ]:
fake.iloc[fake_empty_index]

In [ ]:
## here we can see in fake dataframe 630 text rows are missing. So now for training data we merge title and text columns
real['text'] = real['title'] + " " + real['text']
fake['text'] = fake['title'] + " " + fake['text']

Text Pre-processing

In [ ]:
real['text'] = real['text'].apply(lambda x : str(x).lower())
fake['text'] = fake['text'].apply(lambda x : str(x).lower())

In [ ]:
real['class'] = 1
fake['class'] = 0

In [ ]:
real.columns

In [ ]:
real = real[['text', 'class']]
fake = fake[['text', 'class']]

In [ ]:
cols = [real, fake]
data = pd.concat(cols)
data.sample(5)

In [ ]:
## remove special char from text

# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install beautifulsoup4
# !pip install textblob

In [ ]:
def remove_special_chars(x):
	x = re.sub(r'[^\w ]+', "", x)
	x = ' '.join(x.split())
	return x


In [ ]:
data['text'].apply(lambda x: remove_special_chars(x))

Vectorization - word2vec

In [ ]:

# !pip install gensim

import gensim


In [ ]:
y = data['class'].values

In [ ]:
X = [d.split() for d in data['text']]  ## create the list of list of words from text columns

In [ ]:
DIM = 100
w2v_model = gensim.models.Word2Vec(sentences=X, vector_size=DIM, window=10, min_count=1, sg=1)

In [ ]:
w2v_model.wv['india']

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

In [ ]:
X = tokenizer.texts_to_sequences(X)
print(X)

In [ ]:
tokenizer.word_index  ## convert word to sequence 

In [ ]:
## check the maximum length of the sentance

plt.hist([len(x) for x in X], bins=700)
plt.show()

In [ ]:
maxlen = 1000
X = pad_sequences(X, maxlen=maxlen)

In [ ]:
X

In [ ]:
vocab_size = len(tokenizer.word_index) + 1  ## 1 is for unknown sequence
vocab = tokenizer.word_index

In [ ]:
def get_weight_matrix(model):
    weight_matrix = np.zeros((vocab_size, DIM))

    for word, i in vocab.items():
        weight_matrix[i] = model.wv[word]
        
    return weight_matrix

In [ ]:
embedding_vectors = get_weight_matrix(w2v_model)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
model.add(LSTM(units = 128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
x_train, x_test, y_train,  y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model.fit(x_train, y_train, validation_split=0.3, epochs=6)

In [ ]:
y_pred = (model.predict(x_test) >= 0.5).astype(int)

In [ ]:
y_pred 

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))